## Converting to TRT
Reference :https://hemanths933.medium.com/convert-onnx-bert-model-to-tensorrt-e809276b01b6

In [4]:
#Import Packages
import numpy as np
from onnx_helper import ONNXClassifierWrapper,convert_onnx_to_engine
import torch
import json
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import time
import onnxruntime as onnxrt
%matplotlib inline

In [15]:
gpuid = 0
device = torch.device('cpu') if gpuid < 0 else torch.device(f'cuda:{gpuid}')
print('Using device:', device)

Using device: cuda:0


## Lift Splat

In [10]:
#Set constants

PRECISION=np.float32

TRT_PATH='models/reduction_5_model.trt'

In [5]:
print("Loading TRT")

trt_model=ONNXClassifierWrapper('models/simp_cam_encode_b1.trt',[6, 64, 41, 8, 22],target_dtype=np.float32)



Loading TRT


In [6]:
dummy_batch=torch.ones(6, 3, 128, 352).numpy()
#dummy_batch=dummy_batch.transpose((0,3,2, 1))
print(dummy_batch.shape)
#dummy_batch = np.ascontiguousarray(dummy_batch)
print(dummy_batch.nbytes)

(6, 3, 128, 352)
3244032


In [13]:
start = time.time()
predictions1=trt_model.predict(dummy_batch)
end = time.time()
#print("Prediction",predictions)
print(f"Time taken to eval in the model {(end-start)*1000} ms ")

Time taken to eval in the model 24.132251739501953 ms 


In [18]:
dummy_batch=torch.ones(1, 3, 128, 352, device =device)
ort_session = onnxrt.InferenceSession('models/simp_cam_encode_b1.onnx', None, providers=["CUDAExecutionProvider"])
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
inputs = [node for node in ort_session.get_inputs()][0]
outputs = [node.name for node in ort_session.get_outputs()]

pred_onx4 = ort_session.run(outputs, {inputs.name: to_numpy(dummy_batch)})
print(pred_onx4[0].shape)
np.testing.assert_allclose(predictions1, pred_onx4[0], rtol=1e-03, atol=1e-05)

(1, 64, 44, 8, 22)


AssertionError: 
Not equal to tolerance rtol=0.001, atol=1e-05

(shapes (6, 64, 41, 8, 22), (1, 64, 44, 8, 22) mismatch)
 x: array([[[[[ 4.537674e-04,  3.905300e-04, -3.677754e-05, ...,
           -5.993718e-04,  9.560145e-04, -5.612092e-04],
          [ 2.690700e-03,  2.336699e-03,  1.739836e-03, ...,...
 y: array([[[[[ 3.841190e-04,  3.894048e-04, -6.458013e-06, ...,
           -4.762142e-04,  4.846490e-04, -7.814218e-04],
          [ 2.690520e-03,  2.418729e-03,  1.786525e-03, ...,...

In [28]:
np.linalg.norm(predictions1-pred_onx4[0])

274.0002

## Reference Code

In [ ]:
import tensorrt as trt
import pycuda.autoinit
import pycuda.driver as cuda
def build_engine(model_file, max_ws=512*1024*1024, fp16=False):
    print("building engine")
    TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
    builder = trt.Builder(TRT_LOGGER)
    builder.fp16_mode = fp16
    config = builder.create_builder_config()
    config.max_workspace_size = max_ws
    if fp16:
        config.flags |= 1 << int(trt.BuilderFlag.FP16)
    
    explicit_batch = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
    network = builder.create_network(explicit_batch)
    with trt.OnnxParser(network, TRT_LOGGER) as parser:
        with open(model_file, 'rb') as model:
            parsed = parser.parse(model.read())
            print("network.num_layers", network.num_layers)
            last_layer = network.get_layer(network.num_layers - 1)
            network.mark_output(last_layer.get_output(0))
            engine = builder.build_engine(network, config=config)
            return engine
engine = build_engine("models/camencode_b0.onnx")

building engine


In [ ]:
with open('models/camencode_b0.trt', 'wb') as f:
    f.write(bytearray(engine.serialize()))